# Building a GPT-based Language Model

In this notebook, we will implement a Generative Pre-trained Transformer (GPT)-based language model using PyTorch. The model will be trained on the text data from "The Wizard of Oz" and will be capable of generating new text based on a given prompt.

Objectives
- Understand the basic concepts of self-attention and transformer models.
- Build a GPT architecture from scratch.
- Train the model on a text corpus.
- Generate new text based on a prompt.

In [1]:
# Import libraries
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# Checking If Cuda is available

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Check if Apple's Metal Performance shaders are available
if torch.backends.mps.is_available():
    device = torch.device('mps')
    x = torch.ones(1, device = device)
    print(x)
else:
    print("MPS device not found")

tensor([1.], device='mps:0')


In [3]:
# Hyperparameter for model training

block_size = 128  # Size of the input block
batch_size = 32
max_iters = 25000   # Maximum number of iteration
learning_rate = 3e-3 # Learning rate  
eval_iters = 250 # Number of iterations to perform evaluation
n_embd = 384  # Dimension of the embeddings
n_layer = 4  # Number of layers
n_head = 4   # Number of Attention heads
dropout = 0.2  # Dropout rate

In [4]:
# Read and prepare the text datatset
chars = ""

# Reading in the text file
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

# Vocabulary size
vocab_size = len(chars)

In [5]:
# Create vocabulary mappings
string_to_int = { ch:i  for i, ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Convert entire text to tensor Integer
data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
# Training and Validation Split
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# Function to get batches of data for training/validation
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

## Model Architecture

### **Transformer Model Architecture (GPT):**

1. **Input Embedding:**
   - Converts input tokens (words or subwords) into dense vectors of fixed dimensions. This embedding captures the semantic meaning of the words.

2. **Position Embedding:**
   - Since the Transformer doesn't have a built-in sense of order (unlike RNNs), positional embeddings are added to the input embeddings to provide information about the position of each token in the sequence.

3. **Stacked Layers:**
   - The architecture consists of multiple layers, each containing several key components:

   - **Masked Multi-Head Attention:**
     - Attention mechanisms allow the model to focus on different parts of the input sequence when predicting the next token. Multi-head attention enables the model to capture various relationships between words by processing the information in parallel across multiple heads.
     - Masking ensures that the model only attends to previous tokens when predicting the next token, making the model suitable for autoregressive tasks like text generation.
     The attention mechanism computes a weighted sum of input values (or embeddings) where the weights are determined by the relevance of each input to a particular query. 

   - **Add & Norm (Layer Normalization):**
     - After attention, residual connections (add) are used to pass the original input to the next layer, and layer normalization helps stabilize the training process by normalizing the output.

   - **Feed-Forward Layer:**
     - A fully connected feed-forward neural network applied to each position independently. This layer helps in transforming the data and adding non-linearity.

   - **Add & Norm (Layer Normalization):**
     - Another layer normalization step is applied after the feed-forward network, ensuring that the outputs remain stable.

4. **Linear & Softmax Layer:**
   - After processing through multiple layers, the final layer is a linear transformation that maps the output to the vocabulary size.
   - The softmax function then converts these logits into probabilities, predicting the probability of each token being the next in the sequence.

### Key Characteristics:
- **Self-Attention:** Allows the model to weigh the importance of different tokens in the context of predicting the next token.
- **Autoregressive:** Predicts the next token sequentially by conditioning on previously predicted tokens.
- **Stacked Layers:** Multiple layers allow the model to learn complex patterns and dependencies in the data.

In [7]:
class Head(nn.Module):
    """
    One head of self-attention
    """
    
    def __init__(self, head_size):
        super().__init__()
        # Define linear layers for key, query and value projections
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        
        # Lower triangular matrix for masking
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        # Input shape: (B, T, C) where B = batch size, T = time steps, C = embedding dimension
        B, T, C = x.shape

        # Calculate the key, query, and value vectors (B, T, head_size)
        k = self.key(x) # (B, T, hs)
        q = self.query(x) # (B, T, hs)
        
        # Compute attention scores (Scaled dot-product)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]** -0.5  #(B, T, T)

        # Apply the triangular mask to prevent attending to future positions
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        # softmax to normalize into probabilities
        wei = F.softmax(wei, dim=-1)
        
        wei = self.dropout(wei)
       
        # Compute the weighted sum of the value vectors
        v = self.value(x)
        out = wei @ v 
        return out


class MultiHeadAttention(nn.Module):
    """
    Mulitple heads of self_attention in parallel
    """
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        # Initialize multiple attention heads
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        # Concatenate the outputs from each attention head along the last dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F)
        out = self.dropout(self.proj(out))
        return out
        

class FeedForward(nn.Module):
    """ A simple feedforward network with ReLU non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),            
        )
    
    def forward(self, x):
        return self.net(x)




class Block(nn.Module):
    """
    Transformer block: Multi-head self-attention followed by feedforward network.
    """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the  number of heads we'd like
        super().__init__()
        # Calculate the size of the each attention head
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x




class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Token embedding table for converting tokens to embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # Positional embedding tabl
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Transformer block
        self.blocks = nn.Sequential(*[Block(n_embd, n_head= n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        # Initialize weights for linear and embedding layers
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        # B = batch size, T = sequence length (index contains token indices)
        B, T = index.shape

        # Token embeddings and positional embeddings
        tok_emb = self.token_embedding_table(index) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        
        # Combine token and position embeddings
        x = tok_emb + pos_emb
        
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            
            #focus only on the lasrt time step
            logits = logits[:, -1, :] #becomes (B,C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


In [8]:
model = GPTLanguageModel(vocab_size)  # GPTLanguageModel with specified vocabulary size
m = model.to(device) # Move the model to selected device

In [9]:
@torch.no_grad()
def estimate_loss():
    """
    Estimate the loss for both the training and validation sets.
    """
    out = {}
    model.eval()
    for split in ['train',  'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# Create an optimizer for the model parameters using AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


# Training loop for a specified number of iterations
for iter in range(max_iters):
    
    # Every eval_iters iterations, evaluate and print the training and validation losses
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Print final loss value after training
print(loss.item())

step: 0, train loss: 4.492, val loss: 4.487
step: 250, train loss: 3.147, val loss: 3.155
step: 500, train loss: 3.140, val loss: 3.152
step: 750, train loss: 3.145, val loss: 3.154
step: 1000, train loss: 3.138, val loss: 3.155
step: 1250, train loss: 3.139, val loss: 3.155
step: 1500, train loss: 3.136, val loss: 3.149
step: 1750, train loss: 3.140, val loss: 3.149
step: 2000, train loss: 3.145, val loss: 3.158
step: 2250, train loss: 3.140, val loss: 3.151
step: 2500, train loss: 3.132, val loss: 3.148
step: 2750, train loss: 3.140, val loss: 3.149
step: 3000, train loss: 3.139, val loss: 3.153
step: 3250, train loss: 3.137, val loss: 3.151
step: 3500, train loss: 3.138, val loss: 3.155
step: 3750, train loss: 3.137, val loss: 3.150
step: 4000, train loss: 3.139, val loss: 3.147
step: 4250, train loss: 3.138, val loss: 3.151
step: 4500, train loss: 3.135, val loss: 3.149
step: 4750, train loss: 3.141, val loss: 3.151
step: 5000, train loss: 3.140, val loss: 3.148
step: 5250, train l

In [15]:
filename = 'model-01.pt'

def save_model(model, filename):
  """
  Saves the PyTorch model to a file.

  Args:
    model: The PyTorch model to save.
    filename: The filename to save the model to.
  """
  torch.save(model.state_dict(), filename)
  return


def load_model(filename, vocab_size):
  """
  Loads a PyTorch model from a file.

  Args:
    filename: The filename to load the model from.
    config: The GPTConfig object for the model.

  Returns:
    The loaded PyTorch model.
  """
  model = GPTLanguageModel(vocab_size)
  model.load_state_dict(torch.load(filename, weights_only=True))
  return model

save_model(model, filename)
model = load_model(filename, vocab_size)
model = model.to(device)

In [16]:
prompt = 'Hello can you see me' # Define the initial prompt
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)

# Generate additional tokens based on the initial prompt
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())

print(generated_chars)

Hello can you see me  aheaegdhtb,oieLn0i"oan  r[enfk lmoap toeei   go t  e ohl.'S orec iaoa  da hor reoiiy ai  clle"yrlo
